In [6]:
#Instalar pandas, dnspython
!pip install pandas
!pip install dnspython
# Instalar la librería pymongo, para trabajar con bases de datos no relacionales MongoDB
!pip install pymongo

You should consider upgrading via the 'C:\Users\jaime\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\jaime\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\jaime\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np
# Se usa la librería pymongo para 
import pymongo

In [8]:
#Carga del csv a manipular
accidentes_bicicletas=pd.read_csv("./AccidentesBicicletas_2021.csv", sep=";")

In [9]:
#Limpiamos el dataframe. Todas columnas que no necesitamos a la vez
accidentes_bicicletas_clean = accidentes_bicicletas.drop(labels=['positiva_droga','numero','cod_distrito','tipo_lesividad','coordenada_x_utm','coordenada_y_utm','num_expediente'], axis=1, inplace=False)
accidentes_bicicletas_clean

,fecha,hora,localizacion,distrito,tipo_accidente,estado_meteorológico,tipo_vehículo,tipo_persona,rango_edad,sexo,cod_lesividad,positiva_alcohol
0,01/01/2021,11:38:00,CALL. JOSE BERGAMIN / CALL. FLORENCIO CANO CRI...,MORATALAZ,Caída,Despejado,Bicicleta,Conductor,De 45 a 49 años,Hombre,2.0,N
1,04/01/2021,11:30:00,PLAZA. GRECIA / AVDA. ARCENTALES wanda farola 20,SAN BLAS-CANILLEJAS,Caída,Despejado,Bicicleta,Conductor,De 15 a 17 años,Hombre,14.0,N
2,04/01/2021,11:30:00,PLAZA. GRECIA / AVDA. ARCENTALES wanda farola 20,SAN BLAS-CANILLEJAS,Caída,Despejado,Bicicleta,Conductor,De 18 a 20 años,Hombre,14.0,N
3,04/01/2021,11:30:00,PLAZA. GRECIA / AVDA. ARCENTALES wanda farola 20,SAN BLAS-CANILLEJAS,Caída,Despejado,Bicicleta,Conductor,De 21 a 24 años,Hombre,14.0,N
4,04/01/2021,11:30:00,PLAZA. GRECIA / AVDA. ARCENTALES wanda farola 20,SAN BLAS-CANILLEJAS,Caída,Despejado,Bicicleta,Conductor,De 50 a 54 años,Hombre,6.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...
840,29/12/2021,13:30:00,AVDA. RAMON Y CAJAL / PASEO. CEREZOS,CHAMARTÍN,Caída,Despejado,Bicicleta,Conductor,De 50 a 54 años,Hombre,2.0,N
841,30/12/2021,21:00:00,"CALL. BERRUGUETE, 22",TETUÁN,Caída,Despejado,Bicicleta,Conductor,De 25 a 29 años,Mujer,NaN,N
842,30/12/2021,14:40:00,"CALL. ENTRE ARROYOS, 11",MORATALAZ,Caída,Despejado,Bicicleta,Conductor,De 30 a 34 años,Mujer,14.0,N
843,29/12/2021,23:10:00,CALL. TOMELLOSO / CALL. CRISTO DE LA VEGA,USERA,Colisión fronto-lateral,Despejado,Bicicleta,Conductor,De 21 a 24 años,Hombre,7.0,N


In [10]:
#Operamos en base de datos local
uri_mongo_local="mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"

In [12]:
conexion = pymongo.MongoClient(uri_mongo_local)

In [13]:
#Vemos que bases de datos hay disponibles
todas_bbdd = conexion.list_database_names()
for bbdd in todas_bbdd:
    print(bbdd)

admin
config
local


In [14]:
#Obtenemos (o creamos si no existe) la base de datos bicis madrid
base_datos = conexion.bicismadrid

In [15]:
# Obtenemos (o creamos si no existe) la coleccion accidentes bicicleta
bicismadridaccidents = base_datos.bicismadridaccidents

In [16]:
#Por si queremos limpiar para empezar de cero
#bicismadridaccidents.drop()

In [17]:
#Inserta todos los valores del Dataframe de pandas como documentos en la coleccion bicismadridaccidents
bicismadridaccidents.insert_many(accidentes_bicicletas_clean.to_dict("records"))

In [18]:
#Observamos que la coleccion tiene documentos
print (base_datos.list_collection_names())

['bicismadridaccidents']


In [19]:
#Devuelve algun documento
bicismadridaccidents.find_one()

{'_id': ObjectId('621a155b7057832d0b285ce0'),
 'fecha': '01/01/2021',
 'hora': '11:38:00',
 'localizacion': 'CALL. JOSE BERGAMIN / CALL. FLORENCIO CANO CRISTOBAL',
 'distrito': 'MORATALAZ',
 'tipo_accidente': 'Caída',
 'estado_meteorológico': 'Despejado',
 'tipo_vehículo': 'Bicicleta',
 'tipo_persona': 'Conductor',
 'rango_edad': 'De 45 a 49 años',
 'sexo': 'Hombre',
 'cod_lesividad': 2.0,
 'positiva_alcohol': 'N'}

In [20]:
# Consulta 1:Aquellos accidentes en los que el conductor estaba ebrio (no deberia haber ninguno)
accidentes_con_alcohol = bicismadridaccidents.count_documents({"positiva_alcohol":'Y'})
print("Hubo {} accidentes con alcohol implicado".format(accidentes_con_alcohol))
if accidentes_con_alcohol == 0:
    print("Bien hecho, ciudadanos. Si bebeis no conduzcais. Ni bicis siquiera!")
else:
    print("Hay que hacer algo con los que beben y conducen!")

Hubo 0 accidentes con alcohol implicado
Bien hecho, ciudadanos. Si bebeis no conduzcais. Ni bicis siquiera!


In [21]:
# Otra cosulta: cuántos accidentes hubo en Moratalaz
accidentes_en_moratalaz = bicismadridaccidents.count_documents({'distrito':'MORATALAZ'})
print("En Moratalaz hubo {} accidentes".format(accidentes_en_moratalaz))

En Moratalaz hubo 19 accidentes


In [22]:
# Accidentes que se producen con bicicleta electrica
accidentes_bici_normal = bicismadridaccidents.count_documents({'tipo_vehículo':"Bicicleta"})
accidentes_bici_electrica = bicismadridaccidents.count_documents({'tipo_vehículo':{"$ne" :"Bicicleta"}})
#Porcentaje de bicicletas electricas que tienen accidentes
porcentaje_accidentes_electrica = 100*(accidentes_bici_electrica/bicismadridaccidents.count_documents({}))
print("Hay un {} % de accidentes con bicicleta eléctrica".format(porcentaje_accidentes_electrica))

Hay un 18.224852071005916 % de accidentes con bicicleta eléctrica


In [23]:
#Porcentaje accidentes bicis normales
porcentaje_accidentes_bici_normal = 100*(accidentes_bici_normal/bicismadridaccidents.count_documents({}))
porcentaje_accidentes_bici_normal
print("Hay un {} % de accidentes con bicicleta normal".format(porcentaje_accidentes_bici_normal))

Hay un 81.77514792899409 % de accidentes con bicicleta normal


In [24]:
#Consulta 2: Consultar los 3 distritos con mas accidentes
distritos_por_accidentes = bicismadridaccidents.aggregate([{
    "$group": {
        "_id":"$distrito",
        "numero": {"$sum" : 1}
    }
    },
    {"$sort":
     { "numero": -1 }
    },
    {"$limit":3}
])
print("Los distritos con más accidentes fueron:\n")
for distrito in distritos_por_accidentes:
    print(distrito)


Los distritos con más accidentes fueron:

{'_id': 'CENTRO', 'numero': 102}
{'_id': 'MONCLOA-ARAVACA', 'numero': 77}
{'_id': 'SALAMANCA', 'numero': 66}


In [25]:
#Consulta 3 (parte 1): Encunetra la media de las lesividades
media_de_lesividad = bicismadridaccidents.aggregate([
      {
        "$match":{
            "cod_lesividad": {
                "$exists": True, 
                "$ne": np.nan
            }
        }
    },  
    {"$group": {
        "_id":"_id",
        #"Numero de accidentes": {"$sum" : 1},
        "Media de lesividad" :{"$avg": "$cod_lesividad"}
    }
    },
    {"$sort":
     { "numero": -1 }
    },
    {"$limit":1}
])
for media in media_de_lesividad:
    media_final=media
    
media = media_final['Media de lesividad']
print("La media de lesividad es {}".format(media))

La media de lesividad es 5.756164383561644


In [26]:
#Consulta 3(parte 2): Encuentra aquellos distritos cuya lesvidad sea mayor (a menor grado de lesividad mas probabilidad de ingreso hospitalario)
agregados_lesividad = bicismadridaccidents.aggregate([
      {
        "$match":{
            "cod_lesividad": {
                "$exists": True, 
                "$ne": np.nan,
                "$lt": media
            }
        }
    },  
    {"$group": {
        "_id":"$distrito",
        "Numero de accidentes": {"$sum" : 1},
        "Media de lesividad" :{"$avg": "$cod_lesividad"}
    }
    },
    {"$sort":
     { "Media de lesividad": 1 }
    }
])
print("Los distritos con una media de lesividades más graves fueron:\n")
for agregado in agregados_lesividad:
    print(agregado)
    
print("\n\nNOTA: A menor código de lesividad, mayor gravedad en la heridas sufridas por el accidentado")

Los distritos con una media de lesividades más graves fueron:

{'_id': 'USERA', 'Numero de accidentes': 7, 'Media de lesividad': 1.5714285714285714}
{'_id': 'VILLA DE VALLECAS', 'Numero de accidentes': 3, 'Media de lesividad': 1.6666666666666667}
{'_id': 'PUENTE DE VALLECAS', 'Numero de accidentes': 13, 'Media de lesividad': 1.6923076923076923}
{'_id': 'RETIRO', 'Numero de accidentes': 23, 'Media de lesividad': 1.7391304347826086}
{'_id': 'ARGANZUELA', 'Numero de accidentes': 27, 'Media de lesividad': 1.7407407407407407}
{'_id': 'CENTRO', 'Numero de accidentes': 35, 'Media de lesividad': 1.7428571428571429}
{'_id': 'VILLAVERDE', 'Numero de accidentes': 6, 'Media de lesividad': 1.8333333333333333}
{'_id': 'VICÁLVARO', 'Numero de accidentes': 8, 'Media de lesividad': 1.875}
{'_id': 'BARAJAS', 'Numero de accidentes': 9, 'Media de lesividad': 1.8888888888888888}
{'_id': 'CHAMBERÍ', 'Numero de accidentes': 17, 'Media de lesividad': 1.9411764705882353}
{'_id': 'MONCLOA-ARAVACA', 'Numero de a